In [ ]:
%pip install modal-client
%pip install ipywidgets

In [ ]:
import modal

assert modal.__version__ > "0.49.0"

In [ ]:
import modal

stub = modal.Stub(name="example-basic-notebook")

### Handling standard Python functions

Standard Python functions can of course be defined in a notebook and used on their own or be called within Modal functions.
Below the `double` function is defined in pure-Python, and called once locally.

In [ ]:
def double(x: int) -> int:
    return x + x


double(5)

### Handling Modal Functions

If we wanted to run this trivial doubling function *in the cloud* we can write another function `double_with_modal` and decorate it with `@stub.function` to register
the function with the Modal stub.

To demonstrate that Modal functions you define in the notebook can be called by _other_ Modal functions, there's another function, `quadruple`, which uses `double` and `double_with_modal`.
For numbers greater than 1 million, this function spins up containers that run in Modal, which is a _very_ inefficient way to multiply a number by four, but you can do it if you please!

Be aware that all Modal functions are defined and run using `with stub.run()` in a single cell. Currently, putting all Modal functions in a single-cell is a limitation of the Modal client.
We aim to make notebook code organization more flexible in the future.

In [ ]:
@stub.function()
def double_with_modal(x: int) -> int:
    return x + x


@stub.function()
def quadruple(x: int) -> int:
    if x <= 1_000_000:
        return double(x) + double(x)
    else:
        return double_with_modal.remote(x) + double_with_modal.remote(x)


with stub.run():
    print(quadruple.local(100))
    print(quadruple.remote(100))  # run remotely
    result = quadruple.remote(10_000_000)

In [ ]:
# Evaluate the result created above.
result